<a href="https://colab.research.google.com/github/ebatty/IntroCompNeuro/blob/main/lectures/05_NeuralEncodingII.ipynb" target="_blank"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

# 05 - Neural Encoding II

Coming soon!